In [65]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import rayleigh

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation

import sys

In [66]:
sys.path.append("/home/michael/SBDPP_sim") # your path to the SBDPP_sim folder
import simulation

In [67]:
L = 50.0
M = 4

r_values = np.arange(0, 5.01, 0.01)

# 2D standard normal distribution radial density
# The radial distribution for a 2D normal is proportional to Rayleigh distribution with scale 1 (standard normal case)
def density(r):
    return (1 / (2 * np.pi)) * np.exp(-0.5 * r**2)

radial_density_values = density(r_values)
q_values = np.arange(0, 1.0, 0.001)
inverse_radial_values = rayleigh.ppf(q_values, scale = 1.0)

g2 = simulation.PyGrid2(
    M = M,
    areaLen = [L, L],
    cellCount = [L, L],
    isPeriodic = True,
    birthRates = [1] * M,
    deathRates = [0] * M,
    ddMatrix = [0.1, 0.3, 0.3, 0.3,
                0.3, 0.1, 0.3, 0.3,
                0.3, 0.3, 0.1, 0.3,
                0.3, 0.3, 0.3, 0.1],
    birthX = [q_values.tolist()] * M,
    birthY = [inverse_radial_values.tolist()] * M,
    deathX_ = [[r_values.tolist()]*M] * M,
    deathY_ = [[radial_density_values.tolist()] * M] * M,
    cutoffs = [5.0] * M * M,
    seed = 42,
    rtimeLimit = 7200.0
)


N = 1000  # Number of [x, y] pairs per group

coordinates = []
for _ in range(M):
    group = [[np.random.uniform(0, L), np.random.uniform(0, L)] for _ in range(N)]
    coordinates.append(group)

g2.placePopulation(coordinates)

print("Birth rate:", g2.total_birth_rate)
print("Death rate:", g2.total_death_rate)

Birth rate: 4000.0
Death rate: 1602.1050115717705


In [ ]:
data = list()

for t in (pbar := tqdm(range(30))):
    for c in range(g2.get_num_cells()):
        for n in range(M):
            coords = g2.get_cell_coords(c, n)
            for i in range(len(coords)):
                x,y = coords[i]
                data.append([t, x, y, n])
    g2.run_for(1)    
    pbar.set_description(f"Population: {g2.total_population}, Progress")

Population: 21909, Progress: 100%|██████████| 30/30 [01:59<00:00,  3.98s/it]


In [72]:
df = pd.DataFrame(columns = ['time', 'x', 'y', 's'], data = data)
display(df.set_index('time'))

,x,y,s
time,,,
0,0.800354,0.235753,1
0,0.421223,0.881903,1
0,0.313140,0.376307,1
0,0.940997,0.246175,1
0,0.806012,0.103859,1
...,...,...,...
29,49.565267,49.026213,1
29,49.599531,49.976014,1
29,49.208770,49.580654,2


In [73]:
movie_writer = animation.FFMpegWriter(
    fps = 2,
    codec = 'h264',
    extra_args = ['-vcodec', 'libx264']
)

In [74]:
matplotlib.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg' # Set the path to ffmpeg

df = df.sort_values('time')
output_file = "points_animation.mp4"

fig, ax = plt.subplots(figsize=(8, 8))

# Set axis limits
ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_xlabel('x')
ax.set_ylabel('y')
sc = ax.scatter([], [], c = [], cmap = 'plasma', s = 20, alpha = 0.6)

# Animation function
def update(frame):
    current_df = df[df['time'] == frame]
    sc.set_offsets(current_df[['x', 'y']].values)
    sc.set_array(current_df['s'].values)
    ax.set_title(f"Time: {frame}")
    return sc,

ani = FuncAnimation(fig, update, frames=df['time'].nunique(), blit = True)

# Save as MP4
ani.save(output_file, writer = movie_writer)
print(f"Animation saved as {output_file}")
plt.close(fig)

Animation saved as points_animation.mp4
